In [7]:
# Get the environment variables
from dotenv import load_dotenv
import os

load_dotenv()
print(f"OpenAI API key => {len(os.getenv('OPENAI_API_KEY')) * '#'}")
print(f"Langchain API Key => {len(os.getenv('LANGCHAIN_API_KEY')) * '#'}")
print(f"Langchain project name => {os.getenv('LANGCHAIN_PROJECT_NAME')}")
print(f"Langchain endpoint => {os.getenv('LANGCHAIN_ENDPOINT')}")
print(f"Langchain tracking => {os.getenv('LANGCHAIN_TRACING_V2')}")

print(f"OpenSearch HOST => {os.getenv('OPENSEARCH_HOST')}")
print(f"OpenSearch PORT => {os.getenv('OPENSEARCH_PORT')}")
print(f"OpenSearch index name => {os.getenv('OPENSEARCH_INDEX_NAME')}")
print(f"OpenSearch account ID => {os.getenv('OPENSEARCH_ACCOUNT_ID')}")
print(f"OpenSearch account password => {len(os.getenv('OPENSEARCH_ACCOUNT_PASSWORD')) * '#'}")

OpenAI API key => ########################################################
Langchain API Key => ###################################################
Langchain project name => ex_cti_rag_mitre_002
Langchain endpoint => https://api.smith.langchain.com
Langchain tracking => true
OpenSearch HOST => search-threatghoul-whlfoep64hczlupq5itfzw2cwu.ap-northeast-2.es.amazonaws.com
OpenSearch PORT => 9200
OpenSearch index name => mitreattackragger
OpenSearch account ID => knightchaser
OpenSearch account password => #################


In [8]:
# Start Langsmith tracking
from langsmith import traceable
from langchain.callbacks.tracers import LangChainTracer
from langchain.callbacks.manager import CallbackManager

tracer = LangChainTracer(
    project_name=os.getenv('LANGCHAIN_PROJECT_NAME'),
)
callback_manager = CallbackManager([tracer])

# Wrap OpenAI functions
from langchain_openai import OpenAI
openai_llm = OpenAI(temperature=0, callback_manager=callback_manager)

In [9]:
# Import necessary libraries
from MITREAttackScrapper.cti.groups import MITREAttackCTIGroups
from langchain_core.documents import Document
from typing import List, Dict
from tqdm import tqdm

# Initialize an empty list to store documents
documents: List[Document] = []

# Get the list of MITRE ATT&CK Group IDs
attack_group_id_list: List[str] = [attack_group['id'] for attack_group in MITREAttackCTIGroups.get_list()]

# Iterate over a subset of the attack group IDs to fetch detailed information
for attack_group_id in tqdm(attack_group_id_list):
    attack_group_detail = MITREAttackCTIGroups.get(attack_group_id)
    
    # Extract the details of the threat group
    attack_group_id: str                                    = attack_group_detail['id']
    attack_group_name: str                                  = attack_group_detail['name']
    attack_group_description: str                           = attack_group_detail['description']
    attack_group_contributors: List[str]                    = attack_group_detail.get('contributors', [])
    attack_group_version: str                               = attack_group_detail.get('version', 'N/A')
    attack_group_created: str                               = attack_group_detail.get('created', 'N/A')
    attack_group_last_modified: str                         = attack_group_detail.get('last_modified', 'N/A')
    attack_group_url: str                                   = attack_group_detail['url']
    attack_group_associated_groups: List[Dict[str, str]]    = attack_group_detail.get('associated_group_descriptions', [])
    attack_group_techniques: List[Dict[str, str]]           = attack_group_detail.get('techniques_used', [])
    attack_group_softwares: List[Dict[str, str]]            = attack_group_detail.get('software', [])
    attack_group_references: Dict[int, Dict[str, str]]      = attack_group_detail.get('references', {})

    # Create a description related to the threat group
    description = f"{attack_group_name} (MITRE ATT&CK Group ID: {attack_group_id}) is a threat group that {attack_group_description}.\n"
    
    # Add contributors
    if attack_group_contributors:
        description += f"\nContributors: {', '.join(attack_group_contributors)}\n"
    
    # Add creation and modification dates
    description += f"\nVersion: {attack_group_version}\nCreated: {attack_group_created}\nLast Modified: {attack_group_last_modified}\n"
    
    # Add associated groups
    if attack_group_associated_groups:
        description += "\nAssociated Groups:\n"
        for assoc_group in attack_group_associated_groups:
            description += f"- {assoc_group['name']}: {assoc_group['description']}\n"
    else:
        description += "\nAssociated Groups: None\n"
    
    # Add techniques used
    if attack_group_techniques:
        description += "\nTechniques Used:\n"
        for technique in attack_group_techniques:
            description += (
                f"- {technique['main_technique_name']} (ID: {technique['main_technique_id']}): "
                f"{technique['use']}"
            )
            if 'sub_technique_id' in technique and 'sub_technique_name' in technique:
                description += f" (Sub-technique: {technique['sub_technique_name']} (ID: {technique['sub_technique_id']}))"
            description += "\n"
    else:
        description += "\nTechniques Used: None\n"

    # Add software used
    if attack_group_softwares:
        description += "\nSoftware Used:\n"
        for software in attack_group_softwares:
            description += f"- {software['name']} (ID: {software['id']})"
            if 'techniques' in software:
                description += ", associated techniques: "
                for technique in software['techniques']:
                    # description += f"{technique['name']} (ID: {technique['url'].split('/')[-1]}), "       # Including technique ID is too verbose
                    description += f"{technique['name']}, "
            description += "\n"
    else:
        description += "\nSoftware Used: None\n"

    # Create a document and add it to the list
    document = Document(page_content=description, 
                        metadata={"id": attack_group_id, 
                                  "name": attack_group_name,
                                  "url": attack_group_url,
                                  "reference": attack_group_references})
    documents.append(document)

# The 'documents' list now contains the detailed documents for each MITRE ATT&CK group

100%|██████████| 152/152 [01:02<00:00,  2.45it/s]


In [10]:
print(documents[15])

page_content='APT29 (MITRE ATT&CK Group ID: G0016) is a threat group that APT29 is threat group that has been attributed to Russia's Foreign Intelligence Service (SVR).[1][2] They have operated since at least 2008, often targeting government networks in Europe and NATO member countries, research institutes, and think tanks. APT29 reportedly compromised the Democratic National Committee starting in the summer of 2015.[3][4][5][6].

Contributors: Daniyal Naeem, BT Security, Matt Brenton, Zurich Insurance Group, Katie Nickels, Red Canary, Joe Gumke, U.S. Bank, Liran Ravich, CardinalOps

Version: 6.0
Created: 2017-05-31
Last Modified: 2024-04-12

Associated Groups:
- IRON RITUAL: [15]
- IRON HEMLOCK: [16]
- NobleBaron: [17]
- Dark Halo: [12]
- StellarParticle: [11][18]
- NOBELIUM: [10][19][20][21]
- UNC2452: [9]
- YTTRIUM: [22]
- The Dukes: [3][23][24][13]
- Cozy Bear: [5][23][24][13][18]
- CozyDuke: [5]
- SolarStorm: [14]
- Blue Kitsune: [25][26]
- UNC3524: [27]
- Midnight Blizzard: [28]


In [11]:
# IN CASE OF USING AWS OPENSEARCH(Amazon OpenSearch Service)
from langchain_community.vectorstores import OpenSearchVectorSearch
from langchain_openai import OpenAIEmbeddings
import os
import urllib3
from opensearchpy import OpenSearch, RequestsHttpConnection

# Shut up the SSL warnings
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

# Set up OpenSearch client with fine-grained access control
opensearch_host = os.getenv('OPENSEARCH_HOST')
opensearch = OpenSearch(
    hosts=[{'host': opensearch_host, 'port': 443}],
    http_auth=(os.getenv('OPENSEARCH_ACCOUNT_ID'), os.getenv('OPENSEARCH_ACCOUNT_PASSWORD')),
    use_ssl=True,
    verify_certs=True,
    connection_class=RequestsHttpConnection,
    timeout=300,  # Increase the timeout to 300 seconds
    max_retries=10  # Increase the number of retries
)

# Check if the index exists, and delete it if it does
index_name = os.getenv('OPENSEARCH_INDEX_NAME')
try:
    if opensearch.indices.exists(index=index_name):
        opensearch.indices.delete(index=index_name)
        print(f"Deleted the index {index_name} because it already exists")
    else:
        print(f"The index {index_name} does not exist")
except Exception as e:
    print(f"Error checking or deleting index: {e}")

print(f"Creating an index with the name {index_name}")

# Ensure you have the 'documents' variable defined with your data before this line
try:
    vectorstore = OpenSearchVectorSearch.from_documents(
        index_name=index_name,
        documents=documents,
        embedding=OpenAIEmbeddings(),
        opensearch_url=f"https://{opensearch_host}",
        http_auth=(os.getenv('OPENSEARCH_ACCOUNT_ID'), os.getenv('OPENSEARCH_ACCOUNT_PASSWORD')),
        use_ssl=True,
        verify_certs=True,
        connection_class=RequestsHttpConnection,
        timeout=300,  # Increase the timeout to 300 seconds
        max_retries=10  # Increase the number of retries
    )
    print(f"Index created with {len(documents)} documents")
except Exception as e:
    print(f"Error creating index or adding documents: {e}")


Deleted the index mitreattackragger because it already exists
Creating an index with the name mitreattackragger
Index created with 152 documents


In [ ]:
# IN CASE OF USING OPENSEARCH(Local or not AWSOpenSearch)
from langchain_community.vectorstores import OpenSearchVectorSearch
from langchain_openai import OpenAIEmbeddings
from opensearchpy import OpenSearch
import urllib3

# Shut up the SSL warnings
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

# Delete the index if it exists
opensearch = OpenSearch(
    hosts=[f"{os.getenv('OPENSEARCH_HOST')}:{os.getenv('OPENSEARCH_PORT')}"],
    http_auth=(os.getenv('OPENSEARCH_ACCOUNT_ID'), os.getenv('OPENSEARCH_ACCOUNT_PASSWORD')),
    use_ssl = True,
    verify_certs = False,
    ignore_ssl_warnings = True
)

if opensearch.indices.exists(index=os.getenv('OPENSEARCH_INDEX_NAME')):
    opensearch.indices.delete(index=os.getenv('OPENSEARCH_INDEX_NAME'))
    print(f"Deleted the index {os.getenv('OPENSEARCH_INDEX_NAME')} because it already exists")

print(f"Creating an index with the name {os.getenv('OPENSEARCH_INDEX_NAME')}")

vectorstore = OpenSearchVectorSearch.from_documents(
    index_name=os.getenv('OPENSEARCH_INDEX_NAME'),
    documents=documents,
    embedding=OpenAIEmbeddings(),
    opensearch_url=f"http://{os.getenv('OPENSEARCH_HOST')}:{os.getenv('OPENSEARCH_PORT')}",
    http_auth=(os.getenv('OPENSEARCH_ACCOUNT_ID'), os.getenv('OPENSEARCH_ACCOUNT_PASSWORD')),
    use_ssl = True,
    verify_certs = False,
    ignore_ssl_warnings = True,
)

print(f"Index created with {len(documents)} documents")

In [12]:
# Extract the ontology(top keywords) from the MITRE ATT&CK group information
import nltk
from MITREAttackScrapper.cti.groups import MITREAttackCTIGroups
from typing import Any
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer

# Extract the list of MITRE ATT&CK group information and save it to the cti_group.json
attack_group_information: List[Dict[str, Any]] = MITREAttackCTIGroups.get_list()

# Download NLTK stopwords corpus
nltk.download('stopwords')

# Extract keywords(ontologies) from the descriptions of the MITRE ATT&CK groups, using the TF-IDF method
# Get the top_n keywords from the each description, considering the stopwords and TF-IDF-based importance scoring
def extract_keywords_tfidf(descriptions: List[str], top_n: int = 10) -> Dict[str, List[str]]:
    vectorizer = TfidfVectorizer(stop_words=stopwords.words('english'), max_features=top_n) # Using the English stopwords
    tfidf_matrix = vectorizer.fit_transform(descriptions)                                   # Fit the vectorizer to the descriptions
    feature_names = vectorizer.get_feature_names_out()                                      # Get the feature names
    keywords = {}
    for idx, description in enumerate(descriptions):
        tfidf_scores = tfidf_matrix[idx].toarray()[0]                                       # Get the TF-IDF scores for the description
        top_keywords = [feature_names[i] for i in tfidf_scores.argsort()[-top_n:]]          # Get the top_n keywords
        keywords[attack_group_information[idx]['id']] = top_keywords
    return keywords

descriptions = [group['description'] for group in attack_group_information]
attack_group_ontology = extract_keywords_tfidf(descriptions)

# Print the first five ontology
from pprint import pprint
pprint(dict(list(attack_group_ontology.items())[:5]))

# A function to use ontology for query expansion
def expand_query(query: str, ontology: Dict[str, List[str]]) -> str:
    expanded_terms: List[str] = []
    for group_id, keywords in ontology.items():
        for keyword in keywords:
            if keyword in query.lower():
                expanded_terms.append(group_id)
                break

    expanded_query: str = query + " " + " ".join(expanded_terms)
    return expanded_query

{'G0018': ['active',
           'espionage',
           'government',
           'least',
           'since',
           'group',
           'threat',
           'targeted',
           'cyber',
           'organizations'],
 'G0130': ['government',
           'organizations',
           'targeted',
           'threat',
           'group',
           'since',
           'least',
           'active',
           'cyber',
           'espionage'],
 'G0138': ['espionage',
           'targeted',
           'since',
           'least',
           'threat',
           'active',
           'government',
           'organizations',
           'group',
           'cyber'],
 'G1000': ['active',
           'government',
           'least',
           'organizations',
           'since',
           'threat',
           'targeted',
           'cyber',
           'espionage',
           'group'],
 'G1024': ['cyber',
           'espionage',
           'government',
           'group',
           'organiz

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [13]:
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

# Step 3: Refine Similarity Search
@traceable(name="retrieve_relevant_documents")
def retrieve_relevant_documents(query: str, top_k: int = 3) -> List[Document]:

    # Query Expansion
    expanded_query = expand_query(query, attack_group_ontology)

    # Retrieve relevant documents with 3 times the top_k
    retrieved_documents = vectorstore.similarity_search(expanded_query, k=top_k*3)

    # Embed the expanded query and retrieved documents
    embedding_model = SentenceTransformer("sentence-transformers/multi-qa-MiniLM-L6-cos-v1")
    expanded_query_embedding: List[float] = embedding_model.encode([expanded_query])[0]
    document_embeddings: List[List[float]] = embedding_model.encode([doc.page_content for doc in retrieved_documents])

    # Calculate the cosine similarity between the expanded query and retrieved documents
    cosine_similarities: List[float] = cosine_similarity([expanded_query_embedding], document_embeddings)[0]

    # Combine keyword relevance with semantic similarity
    keyword_matches = [sum(keyword in doc.page_content for keyword in expanded_query.split()) for doc in retrieved_documents]
    combined_scores = [cosine + keyword * 0.1 for cosine, keyword in zip(cosine_similarities, keyword_matches)]

    # Sort the documents based on combined scores and return the top_k documents
    sorted_documents = [doc for _, doc in sorted(zip(combined_scores, retrieved_documents), reverse=True)]
    return sorted_documents[:top_k]

# Example usage
user_question = "Which groups are based on the North Korean threat actor?"
relevant_docs = retrieve_relevant_documents(user_question)

print(f"Retrieved {len(relevant_docs)} relevant documents:")
for index, document in enumerate(relevant_docs):
    print(f"\nDocument {index + 1}:")
    print(f"Content: {document.page_content[:200]}...") 
    print(f"Metadata: {document.metadata}")

c:\Users\user\Documents\GitHub\MITREAttackRagger\venv\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


Retrieved 3 relevant documents:

Document 1:
Content: Lazarus Group (MITRE ATT&CK Group ID: G0032) is a threat group that Lazarus Group is a North Korean state-sponsored cyber threat group that has been attributed to the Reconnaissance General Bureau.[1]...
Metadata: {'id': 'G0032', 'name': 'Lazarus Group', 'url': 'https://attack.mitre.org/groups/G0032/', 'reference': {'1': {'text': 'US-CERT. (2017, June 13). Alert (TA17-164A) HIDDEN COBRA – North Korea’s DDoS Botnet Infrastructure. Retrieved July 13, 2017.', 'url': 'https://www.us-cert.gov/ncas/alerts/TA17-164A'}, '2': {'text': 'US Treasury . (2019, September 13). Treasury Sanctions North Korean State-Sponsored Malicious Cyber Groups. Retrieved September 29, 2021.', 'url': 'https://home.treasury.gov/news/press-releases/sm774'}, '3': {'text': 'Novetta Threat Research Group. (2016, February 24). Operation Blockbuster: Unraveling the Long Thread of the Sony Attack. Retrieved February 25, 2016.', 'url': 'https://web.archive.org/web/201602

In [14]:
import re

# Realign the references of the documents
# References are stored as a dictionary at the metadata of each retrieved document.
# Merge the references of the retrieved documents as the multiple retrieved document set is merged into a single document.
def realign_retrieved_documents(retrieved_documents: List[Document]) -> Dict[str, Any]:
    contents: str = ""
    references: Dict[str, Any] = {}
    reference_number: int = 1               # Start reference numbering from 1
    document_number: int = 1

    for document in retrieved_documents:
        content = document.page_content
        new_content = content

        # Replace the reference marks in the content with new unique numbers
        ref_indices = re.findall(r'\[(\d+)\]', content)
        for ref_index in ref_indices:
            original_ref_number = int(ref_index)
            if original_ref_number not in references:
                references[reference_number] = document.metadata['reference'][str(original_ref_number)]
                new_content = new_content.replace(f'[{original_ref_number}]', f'[{reference_number}]')
                reference_number += 1

        # Append the modified content to the final contents
        contents += f"Retrieved document {document_number}:\n"
        contents += new_content + "\n\n"

    return {
        "contents": contents,
        "references": references
    }

pprint(realign_retrieved_documents(relevant_docs))

{'contents': 'Retrieved document 1:\n'
             'Lazarus Group (MITRE ATT&CK Group ID: G0032) is a threat group '
             'that Lazarus Group is a North Korean state-sponsored cyber '
             'threat group that has been attributed to the Reconnaissance '
             'General Bureau.[1][2] The group has been active since at least '
             '2009 and was reportedly responsible for the November 2014 '
             'destructive wiper attack against Sony Pictures Entertainment as '
             'part of a campaign named Operation Blockbuster by Novetta. '
             'Malware used by Lazarus Group correlates to other reported '
             'campaigns, including Operation Flame, Operation 1Mission, '
             'Operation Troy, DarkSeoul, and Ten Days of Rain.[3].\n'
             '\n'
             'Contributors: Kyaw Pyiyt Htet, @KyawPyiytHtet, Dragos Threat '
             'Intelligence\n'
             '\n'
             'Version: 4.0\n'
             'Created: 2017-05-

In [23]:
# Generate a few-shot prompt template for few-shot learning
from langchain_core.prompts.few_shot import FewShotPromptTemplate
from langchain_core.prompts.prompt import PromptTemplate
from typing import List, Dict

# Define the few-shot examples
examples: List[Dict[str, str]] = [
    {
        "contexts": "Retrieved document 1:\n"
                    "  - Group A is a threat group that targets the financial sector[1][2] since 2018 in United States[3].\n"
                    "    In 2018, they targeted a bank in New York[4].\n"
                    "  - Group A additionally targeted a bank in California[5], company in Texas[6], resulting in a loss of $2 million[7].\n"
                    "Retrieved document 2:\n"
                    "  - Group B is a threat group that targets the private enterprise banking sector[5][6] since 2019 in United States[7].\n"
                    "    In 2019, they targeted a bank in California[8], company in Texas[9], resulting in a loss of $1 million[10].\n"
                    "  - Group B is currently on the list of FBI's most wanted cybercriminals[11], with $1 million bounty[12].",
        "question": "Which groups usually target the financial sector?",
        "answer": """
{{
    "1, 2, 3": "Group A targets the financial sector since 2018 in the United States.",
    "5, 6, 7": "Group B targets the private enterprise banking sector since 2019 in the United States.",
    "[general]": "Both groups have caused significant financial losses in the targeted sectors."
}}
"""
    },
    {
        "contexts": "Retrieved document 1:\n"
                    "  - Group A is a threat group that targets the video-game sector[1][2] since 2018 in United States[3].\n"
                    "    In 2018, they targeted a game development company in California[4].\n"
                    "Retrieved document 2:\n"
                    "  - Group B is a threat group that targets the video-game sector[5][6] since 2019 in United States[7].\n"
                    "    FBI has identified them as a North Korean threat actor[8], and they targeted a game development company in Texas[9].\n"
                    "    Importantly, some of Group B member has been worked while hiding their identities, revealed and arrested by FBI in 2020[10].\n",
        "question": "Which groups are based on the North Korean threat actor and what happened in 2020?",
        "answer": """
{{
    "8": "Group B is identified as a North Korean threat actor.",
    "10": "Some of Group B members were revealed and arrested by FBI in 2020 while working under hidden identities.",
    "[general]": "Group B has been actively involved in the video-game sector targeting."
}}
"""
    }
]

example_prompt = PromptTemplate(
    input_variables=["contexts", "question", "answer"],
    template="Example Context: {contexts}\nExample Question: {question}\nExample Answer: {answer}\n",
)

print(example_prompt.format(**examples[0]))

Example Context: Retrieved document 1:
  - Group A is a threat group that targets the financial sector[1][2] since 2018 in United States[3].
    In 2018, they targeted a bank in New York[4].
  - Group A additionally targeted a bank in California[5], company in Texas[6], resulting in a loss of $2 million[7].
Retrieved document 2:
  - Group B is a threat group that targets the private enterprise banking sector[5][6] since 2019 in United States[7].
    In 2019, they targeted a bank in California[8], company in Texas[9], resulting in a loss of $1 million[10].
  - Group B is currently on the list of FBI's most wanted cybercriminals[11], with $1 million bounty[12].
Example Question: Which groups usually target the financial sector?
Example Answer: 
{{
    "1, 2, 3": "Group A targets the financial sector since 2018 in the United States.",
    "5, 6, 7": "Group B targets the private enterprise banking sector since 2019 in the United States.",
    "[general]": "Both groups have caused significa

In [24]:
# Based on the few-shot example, generate the FewShotPromptTemplate
few_shot_prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    suffix="You are a security analyst and you are asked to identify the threat groups based on the given context and question.\n"
           "Based on the context (retrieved documents) with reference marks such as [1], [2], ... [number] and question, please provide detailed and analytical answers.\n"
           "Note that the reference marks are not MITRE ATT&CK technique/group/... IDs such as T1234, G1234, etc. Only consider reference numbers like [1] for generating responses.\n"
           "You have to use the reference numbers within the retrieved documents to answer the given question.\n"
           "Provide the answer in the format where each part of the answer is mapped to the corresponding references.\n"
           "If the answer is not clear or if there are sentences that are general or don't have a specific reference, mark them as '[general]'.\n"
           "If the given context doesn't provide enough information, mention that as well (e.g., 'Not enough information to answer the given question').\n"
           "Refer to the given few-shot examples to understand the answer format, and provide the answer in the same format.\n"
           "\n\n"
           "Contexts: {contexts}\n"
           "Question: {question}\n"
           "Answer:",
    input_variables=["question", "contexts"]
)

print(few_shot_prompt.format(question="Which groups are based on the North Korean threat actor?",
                             contexts="Example context 1"))

Example Context: Retrieved document 1:
  - Group A is a threat group that targets the financial sector[1][2] since 2018 in United States[3].
    In 2018, they targeted a bank in New York[4].
  - Group A additionally targeted a bank in California[5], company in Texas[6], resulting in a loss of $2 million[7].
Retrieved document 2:
  - Group B is a threat group that targets the private enterprise banking sector[5][6] since 2019 in United States[7].
    In 2019, they targeted a bank in California[8], company in Texas[9], resulting in a loss of $1 million[10].
  - Group B is currently on the list of FBI's most wanted cybercriminals[11], with $1 million bounty[12].
Example Question: Which groups usually target the financial sector?
Example Answer: 
{
    "1, 2, 3": "Group A targets the financial sector since 2018 in the United States.",
    "5, 6, 7": "Group B targets the private enterprise banking sector since 2019 in the United States.",
    "[general]": "Both groups have caused significan

In [25]:
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import JsonOutputParser
from typing import Dict, Any, Union, List, Tuple
import json
import re

# Initialize the GPT-4 model
llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)

# Function to extract and sort used references from the "answer"
def extract_used_references(answer: Dict[str, str]) -> List[str]:
    used_references = set()
    for key in answer.keys():
        if key != "[general]":
            used_references.update(key.split(', '))
    return sorted(used_references, key=int)

# Function to filter and realign references
def realign_references(answer: Dict[str, str], references: Dict[int, Any]) -> Tuple[Dict[str, str], Dict[str, Any]]:
    used_references = extract_used_references(answer)
    new_reference_mapping = {old: str(new) for new, old in enumerate(used_references, 1)}

    # Update references dictionary
    new_references = {new_reference_mapping[old]: references[int(old)] for old in used_references}

    # Update answer dictionary
    new_answer = {}
    for key, value in answer.items():
        if key == "[general]":
            new_answer[key] = value
        else:
            new_key = ', '.join(new_reference_mapping[ref] for ref in key.split(', '))
            new_answer[new_key] = value

    return new_answer, new_references

# Extract the JSON output from the given text
def extract_json_output(text: str) -> Dict[str, Union[Dict[str, str], Any]]:
    if isinstance(text, dict):
        # Convert the dictionary to JSON
        text = json.dumps(text)
    json_regex_pattern = re.compile(r'\{.*\}', re.DOTALL)
    json_regex_match = json_regex_pattern.search(text)

    if json_regex_match:
        json_output = json_regex_match.group(0)
        try:
            # Parse the JSON
            data = json.loads(json_output)
            return data
        except json.JSONDecodeError:
            raise ValueError(f"Invalid JSON format: {json_output}")
    else:
        raise ValueError(f"No JSON output found in the text: {text}")

# Function to generate the answer using the LLM with JSON output
@traceable(name="rag_qa_chain")
def rag_qa_chain(query: str) -> Dict[str, Union[Dict[str, str], Any]]:
    # Retrieve relevant documents
    relevant_docs = retrieve_relevant_documents(query)

    # Realign retrieved documents to handle references
    realigned_docs = realign_retrieved_documents(relevant_docs)

    # Create the chain with JSONOutputParser
    chain = (
        {"contexts": lambda x: realigned_docs["contents"], "question": lambda x: query}
        | few_shot_prompt
        | llm
        | JsonOutputParser()
    )
    
    # Run the chain and remove characters that are not JSON serializable from the response
    llm_answer = chain.invoke(query)
    print(f"LLM answer: {llm_answer}")
    
    # If the answer is not a complete python dictionary (JSON), extract the JSON output
    try:
        _ = llm_answer.keys()
    except AttributeError:
        llm_answer = extract_json_output(llm_answer)
    
    # Extract the "answer" and "references" from the LLM response
    references = realigned_docs["references"]
    print(f"references: {references}")

    # Realign the references and update the answer
    new_answer, new_references = realign_references(llm_answer, references)
    
    # Combine answer with references
    return {
        "answer": new_answer,
        "references": new_references
    }

# Example usage
user_question = "Which techniques are used by the North Korean threat actor? Enumerate all the techniques they used."
answer = rag_qa_chain(user_question)

print(f"User question: {user_question}")
print(f"Answer: {json.dumps(answer, indent=2)}")

LLM answer: {'1': 'Lazarus Group has used Access Token Manipulation (ID: T1134) to obtain user tokens from interactive sessions.', '2': "Lazarus Group has conducted Account Discovery (ID: T1087) to query compromised victim's active directory servers.", '3': 'Lazarus Group has employed Account Manipulation (ID: T1098) to rename the administrator’s account.', '4': 'Lazarus Group has acquired Infrastructure (ID: T1583) by registering domains related to their campaigns.', '5': 'Lazarus Group has used Adversary-in-the-Middle (ID: T1557) to harvest credentials.', '6': 'Lazarus Group has conducted C2 over Application Layer Protocol (ID: T1071) using HTTP and HTTPS.', '7': 'Lazarus Group has archived collected data using Archive Collected Data (ID: T1560) with RAR and ZIP formats.', '8': 'Lazarus Group has maintained persistence through Boot or Logon Autostart Execution (ID: T1547) by adding malicious code to startup folders.', '9': 'Lazarus Group has used Command and Scripting Interpreter (ID

ValueError: invalid literal for int() with base 10: '[general]'

In [ ]:
# TODO: Further improvements; CoVE(Chain of VEritables) for the answer verification